<a href="https://colab.research.google.com/github/ZhuxinJiang/Gen-ai-course/blob/main/lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries for transformers, acceleration, Hugging Face Hub, and 4-bit quantization.
!pip -q install "transformers>=4.45.0" "accelerate>=0.33.0" "huggingface_hub>=0.24.0" bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 16.0 MB/s eta 0:00:00


In [4]:
from google.colab import userdata
from huggingface_hub import login

# Retrieve the Hugging Face token from Colab's user data secrets.
# This ensures sensitive API keys are not hardcoded in the notebook.
hf_token = userdata.get("HF_TOKEN")

# Check if the token was successfully retrieved. If not, raise an error.
# The user needs to add 'HF_TOKEN' to Colab Secrets (the key icon on the left panel).
if not hf_token:
    raise ValueError("HF_TOKEN not found. Add it in Colab Secrets (🔑) and rerun.")

# Log in to the Hugging Face Hub using the retrieved token.
# This authenticates the session, allowing access to models and datasets.
login(token=hf_token)

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Define the model identifier for the Llama 3.2 3B Instruct model.
# This ID is used to fetch the pre-trained model and its tokenizer from Hugging Face.
model_id = "Qwen/Qwen2.5-1.5B"

# Configure BitsAndBytes for 4-bit quantization.
# This helps reduce memory usage by loading the model in 4-bit precision.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable 4-bit quantization.
    bnb_4bit_compute_dtype=torch.float16, # Set the compute data type to float16 for performance.
)

# Load the tokenizer for the specified model.
# The tokenizer is responsible for converting text into numerical tokens that the model can understand.
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

# Load the pre-trained causal language model.
# device_map="auto" automatically distributes the model across available devices (e.g., GPU).
# quantization_config applies the BitsAndBytes configuration for memory-efficient loading.
# torch_dtype=torch.float16 ensures the model is loaded with half-precision floating-point numbers.
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [8]:
def base_chat_once(user_text, transcript: str):
    # Append the user's input and a placeholder for the assistant's response to the transcript.
    transcript += f"\nUser: {user_text}\nAssistant:"

    # Tokenize the entire transcript and move it to the model's device (e.g., GPU).
    inputs = tokenizer(transcript, return_tensors="pt").to(model.device)

    # Generate a response from the model without computing gradients (for inference efficiency).
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=256,  # Maximum number of new tokens to generate for the response.
            do_sample=True,      # Enable sampling to introduce randomness and diversity in responses.
            temperature=0.7,     # Controls the randomness of the output. Lower values make output more deterministic.
            top_p=0.9,           # Filters out low probability tokens, focusing on a smaller set of high probability tokens.
        )

    # Extract only the newly generated tokens from the output_ids.
    new_tokens = output_ids[0][inputs["input_ids"].shape[-1]:]
    # Decode the new tokens back into human-readable text, removing special tokens.
    assistant_text = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

    # Update the transcript with the assistant's generated response.
    transcript += " " + assistant_text
    return assistant_text, transcript


# Initialize the conversation transcript with a system prompt to set the assistant's persona.
transcript = "You are a helpful teaching assistant for a university GenAI applications course.\n"
print("IN6241 Lab 2 — type 'exit' to stop.\n")

# Start an interactive chat loop.
while True:
    user_text = input("You: ") # Get user input from the console.
    # Check if the user wants to end the chat.
    if user_text.lower().strip() in ("exit", "quit"):
        print("Chat ended.")
        break # Exit the loop if user types 'exit' or 'quit'.

    # Get the assistant's response and update the transcript.
    assistant_text, transcript = base_chat_once(user_text, transcript)
    print("\nAssistant:", assistant_text, "\n") # Print the assistant's response.

IN6241 Lab 2 — type 'exit' to stop.

You: HELLO


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



Assistant: Hello! How can I help you with? 

You: exit
Chat ended.
